In [1]:
import torch
import torch.nn as nn
from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
bc = datasets.load_breast_cancer()
X = bc.data
y = bc.target

In [3]:
print(X.shape)

(569, 30)


In [4]:
print(y.shape)

(569,)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [6]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [7]:
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

In [8]:
y_train = y_train.view(y_train.shape[0],1)
y_test = y_test.view(y_test.shape[0],1)

In [9]:
print(y_train.shape,y_test.shape)

torch.Size([426, 1]) torch.Size([143, 1])


In [10]:
nSamples , nFeatures = X.shape

## Model Initialization

In [31]:
class LogisticRegression(nn.Module):
    def __init__(self,input_features):
        super(LogisticRegression,self).__init__()
        self.linear = nn.Linear(input_features,1)
        
    def forward(self,x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted

In [32]:
model = LogisticRegression(nFeatures)

In [33]:
learning_rate = 0.01
loss = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate)

## Training Iterations

In [34]:
epochs = 1000
print("[INFO] Training started...")
for epoch in range(epochs):
    y_pred = model(X_train)
    
    l = loss(y_pred,y_train)
    
    l.backward()
    
    optimizer.step()
    
    optimizer.zero_grad()
    
    if (epoch +1)%100==0:
        print(f"[INFO] Epoch : {epoch}, loss: {l.item()}")

[INFO] Training started...
[INFO] Epoch : 99, loss: 0.26234349608421326
[INFO] Epoch : 199, loss: 0.19499917328357697
[INFO] Epoch : 299, loss: 0.16529785096645355
[INFO] Epoch : 399, loss: 0.1478390097618103
[INFO] Epoch : 499, loss: 0.1360911726951599
[INFO] Epoch : 599, loss: 0.12752072513103485
[INFO] Epoch : 699, loss: 0.12092319130897522
[INFO] Epoch : 799, loss: 0.11564697325229645
[INFO] Epoch : 899, loss: 0.11130577325820923
[INFO] Epoch : 999, loss: 0.10765447467565536


## Test on test_data

In [35]:
print("[INFO] Testing Accuracy...")

with torch.no_grad():
    y_pred = model(X_test)
    y_pred = y_pred.round()
    acc = y_pred.eq(y_test).sum()/float(y_test.shape[0])
    print(f"[INFO] Accuracy : {acc:.4f}")

[INFO] Testing Accuracy...
[INFO] Accuracy : 0.9860
